In [11]:
import cv2
import numpy as np

img = cv2.imread('image/fish.jpg')
img = cv2.resize(img, (680, 420))
rows, cols = img.shape[:-1]

In [12]:
pts1 = np.float32([[0,0], 
                   [0,rows], 
                   [cols,0], 
                   [cols,rows]])

pts2 = np.array([[100,50], 
                [10,rows-50], 
                [cols-100,50], 
                [cols-10,rows-50]], dtype='float32')

In [13]:
cv2.imshow('IMG', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
cv2.circle(img, (0,0), 10, (255,0,0), -1)
cv2.circle(img, (0,rows), 10, (0,255,0), -1)
cv2.circle(img, (cols,0), 10, (0,0,255), -1)
cv2.circle(img, (cols,rows), 10, (0,255,255), -1)

array([[[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]],

       ...,

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255, 255],
        [  0, 255, 255],
        [  0, 255, 255]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255, 255],
        [  0, 255, 255],
        [  0, 255, 255]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255, 255],
        [  0, 255, 255],
        [  0, 255, 255]]

In [15]:
M = cv2.getPerspectiveTransform(pts1, pts2)

In [16]:
dst = cv2.warpPerspective(img, M, (cols, rows))
cv2.imshow('IMG', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
import cv2
import numpy as np

title = "Scanner"
img = cv2.imread('image/paper.jpg')
cv2.imshow(title, img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [36]:
import cv2
import numpy as np

title = "Scanner"
img = cv2.imread("image/paper.jpg")

# 2차원으로 원근적용할 전체 크기
rows, cols = img.shape[:-1]
img2 = img.copy() # 원근변환이 적용될 이미지
pts_cnt = 0 # 필요한 4개의 포인트인지 확인하기 위한 변수
pts = np.zeros((4,2), dtype=np.float32) # 마우스로 클릭할 위치에 대한 변환 행렬

# 마우스 클릭에 사용할 콜백함수
def onMouse(event, x, y, flags, param):
    global pts_cnt
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img2, (x, y), 10, (0,255,255), -1) # 클릭시 이미지에 원 그리기
        cv2.imshow(title, img2)

        pts[pts_cnt] = [x, y] # 마우스 버튼 좌표 저장
        pts_cnt += 1 # 마우스 버튼 클릭 횟수 업데이트

        if pts_cnt == 4:
            # 각 좌표의 위치(상하좌우) 찾는 코드
            sm = pts.sum(axis=1) # x+y
    # diff는 차분(차이값, 변이값)을 사용하는데 시계열 데이터를 사용해서
    # 순차적인 것이 특징이라 앞 데이터가 뒤 데이터에 영향을 줌
    # 그래서 diff는 뒤 데이터에서 앞 데이터를 빼는 함수이다
            # diff = np.diff(pts, axis=1) 
            # x-y여야 하는데 y-x가 됨
            diff = pts[:, 0] - pts[:, 1]

            top_left = pts[np.argmin(sm)]
            top_right = pts[np.argmax(diff)]
            bottom_left = pts[np.argmin(diff)]
            bottom_right = pts[np.argmax(sm)]

            pts1 = np.float32([top_left, top_right, bottom_right, bottom_left])
            print(pts1)
            
            # 각 꼭지점들의 거리 계산
            w1 = abs(bottom_left[0]-bottom_right[0])
            w2 = abs(top_left[0]-top_right[0])
            h1 = abs(top_right[1]-bottom_right[1])
            h2 = abs(top_left[1]-bottom_left[1])
            width = max([w1, w2])
            height = max([h1, h2])
            print(width, height)

            # 변환하는 위치의 값
            pts2 = np.float32([[0,0], [width-1,0],
                                [width-1, height-1], [0,height-1]])
            # 변환 행렬 계산
            M = cv2.getPerspectiveTransform(pts1,pts2)
            # 변환 행렬 적용
            res = cv2.warpPerspective(img, M, (int(width), int(height)))
            cv2.imshow("Result", res)

cv2.imshow(title, img)
cv2.setMouseCallback(title, onMouse)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[101.  53.]
 [497. 101.]
 [569. 696.]
 [ 23. 704.]]
546.0 651.0


In [22]:
pts = np.zeros((4,2), dtype=np.float32)
pts[0] = [10,20]
pts[1] = [5,5]
pts.sum(axis=1)

array([30., 10.,  0.,  0.], dtype=float32)

In [1]:
import cv2
import numpy as np

title = "Scanner"
img = cv2.imread("image/cat1.jpg", cv2.IMREAD_GRAYSCALE)
filter = np.full(img.shape, 15, dtype=np.uint8)
img2 = img + filter

cv2.imshow(title, img)
cv2.imshow("cat", img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
img = cv2.imread("image/cat1.jpg", cv2.IMREAD_GRAYSCALE)
img2 = img * 0.01

print(img.max(), img2.max())
cv2.imshow(title, img)
cv2.imshow("cat", img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

255 2.5500000000000003


In [4]:
import cv2
import numpy as np

img = cv2.imread('image/cat1.jpg')
blur_img = cv2.blur(img, (10,10))
blur = cv2.boxFilter(img, -1, (20,20))

merge_img = np.hstack((img, blur_img, blur))
cv2.imshow('BLUR', merge_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
import cv2
import numpy as np

img = cv2.imread('image/man_face.jpg')
blur_img = cv2.blur(img, (10, 10))
blur = cv2.boxFilter(img, -1, (20,20))

merge_img = np.hstack((img, blur_img, blur))
cv2.imshow('BLUR', merge_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
import cv2
import numpy as np

kernel_size = (40, 40) # 블러 처리에 사용할 커널 크기
title = 'My Self Camera'
img = cv2.imread('image/self.jpg')
img = cv2.resize(img, (680, 960))

while True:
    x,y,w,h = cv2.selectROI(title, img, False) # 관심 영역 선택
    if w > 0 and h > 0: # 폭과 높이가 음수이면 드래그 방향이 옳음
        roi = img[y:y+h, x:x+w] # 관심 영역 지정
        roi = cv2.blur(roi, kernel_size) # 블러(모자이크) 처리
        img[y:y+h, x:x+w] = roi # 원본 이미지에 적용        
        cv2.imshow(title, img)
    else:
        break

cv2.destroyAllWindows()

In [ ]:
# 웹캠에서 블러 처리하는 코드
cap = cv2.VideoCapture(0)
title = "Web Cam"

mask = cv2.imread('image/mask.jpg')
mask = cv2.resize(mask, (150,150))

if cap.isOpened():
    while True:
        ret, img = cap.read()
        if ret:
            x,y,w,h = ((img.shape[0]//2), (img.shape[1]//2), 100, 100)
            roi = img[y:y+h, x:x+w]
            roi = cv2.blur(roi, kernel_size) 
            img[y:y+h, x:x+w] = roi 
            img[y:y+h, x:x+w] = mask # 마스크를 블러 대신 넣는 것
            cv2.imshow(title, img)
            if cv2.waitKey(1) != -1:
                break
        else:
            break

cap.release()
cv2.destroyAllWindows()

In [10]:
# 미분 필터
import cv2
import numpy as np

img = cv2.imread('image/sudoku.jpg')
img = cv2.resize(img, (600, 600))

# 미분 커널 생성(x축, y축)
gx_kernel = np.array([[-1,1]])
gy_kernel = np.array([[-1],
                      [1]])

# 필터 적용(-1은 기본사이즈 사용하겠다는 뜻)
edge_gx = cv2.filter2D(img, -1, gx_kernel) *4
edge_gy = cv2.filter2D(img, -1, gy_kernel) *4

# 결과 출력(x축은 세로의 변화만, y축은 가로의 변화만 찾음)
merge_img = np.hstack((img, edge_gx, edge_gy))
cv2.imshow('edge', merge_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
# 라플라시안 필터
import cv2
import numpy as np

img = cv2.imread('image/sudoku.jpg')
img = cv2.resize(img, (600, 600))

# 라플라시안 필터 적용
edge = cv2.Laplacian(img, -1) * 4

# 결과 출력(x축과 y축 상관없이 경계 전부를 찾음)
merge_img = np.hstack((img, edge))
cv2.imshow('edge', merge_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
# 웹캠에서 필터 적용하는 코드
import cv2
import numpy as np

img = cv2.imread('image/cat2.jpg')
# cap = cv2.VideoCapture(0)
# while cap.isOpened():
    # 프레임 읽기
    # ret, frame = cap.read()
    # 속도 향상을 위해 영상크기를 절반으로 축소
    # frame = cv2.resize(frame, None, fx=0.5, fy=0.5, \
                        # interpolation=cv2.INTER_AREA)
    # if cv2.waitKey(1) == 27: # esc 키로 종료
        # break

# 그레이 스케일로 변경
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 잡음 제거를 위해 가우시안 블러 필터 적용(정규분포를 따라서 블러 처리)
# 이미지를 명확하게 해주기 위해서 사용
img_gray = cv2.GaussianBlur(img_gray, (9,9), 0)
# 라플라시안 필터로 엣지 거출
edge = cv2.Laplacian(img_gray, -1, None, 5)
# 스레시홀드로 경계 값만 남기고 제거하면서 화면 반전
ret, sketch = cv2.threshold(edge, 70, 255, cv2.THRESH_BINARY_INV)

# 경계선을 강조하기 위한 값의 팽창 연산
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
sketch = cv2.erode(sketch, kernel)
# 경계선 자연스럽게 하기 위해 미디언 블러 필터 적용
sketch = cv2.medianBlur(sketch, 5)
# 그레이 스케일에서 BGR 컬러 스케일로 변경
img_sketch = cv2.cvtColor(sketch, cv2.COLOR_GRAY2BGR)

# 컬러 이미지 선명선을 없애기 위해 평균 블러 필터 적용
img_paint = cv2.blur(img, (10,10))
# 컬러 영상과 스케치 영상과 합성
img_paint = cv2.bitwise_and(img_paint, img_paint, mask=sketch)

# 결과 출력
merge_img = np.hstack((img_sketch, img_paint))
cv2.imshow('Sketch Camera', merge_img)

# cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()